In [6]:
import numpy as np
from qiskit import QuantumCircuit,transpile
from qiskit_aer import AerSimulator
# XOR dataset
X = np.array([[0,0],[0,1],[1,0],[1,1]])  
y = np.array([0,1,1,0])

# Parametrized ansatz + feature encoding
def build_circuit(x, params):
    """
    x: input array of 2 features [x1,x2]
    params: array of 4 parameters
    """
    qc = QuantumCircuit(2, 1) # Creates a quantum circuit with 2 qubits(qubit 0, qubit 1) and 1 classical bit(class level) for measurement.
    
    # Encode classical data using RX gates
    for i, val in enumerate(x):  #If a feature is 1, apply an X gate (NOT gate) to flip the qubit.This encodes the classical input into the quantum state.
        if val == 1:
            qc.x(i)
    
    # Ansatz (trainable rotations)
    qc.ry(params[0], 0) #RY(theta) → rotation around Y-axis by theta. These are trainable parameters.
    qc.ry(params[1], 1)
    qc.cx(0,1)          #CX(0,1) → CNOT gate entangles the qubits.
    qc.ry(params[2], 0)  #The second pair of RY gates allows the circuit to adjust its output based on the entanglement.
    qc.ry(params[3], 1)
    
    # Measure qubit 1 (output qubit)
    qc.measure(1,0)   #Measures qubit 1 and stores the result in classical bit 0.  
                      #This is the “output” qubit, whose probability of being 1 will be used as the prediction
    return qc

#Backend
shots = 1024                  # the number of times the circuit is executed.
simulator = AerSimulator()    # simulates the quantum circuit on a classical computer.

#Dummy optimizer loop
params = np.random.rand(4)  # params → initial random parameters  ,4 random parameters for the RY gates.
learning_rate = 0.1  # learning_rate → controls how much we update the parameters each step.

for epoch in range(100):
    print(f"Epoch {epoch+1}")
    for xi, yi in zip(X, y):
        qc = build_circuit(xi, params)  #Build the quantum circuit with build_circuit(xi, params).
        qc_t = transpile(qc,simulator)  #Transpile it for optimization (qc_t = transpile(...)).
        job = simulator.run(qc_t, backend=simulator, shots=shots) # Run the circuit on the simulator
        result = job.result()
        counts = result.get_counts() #Get the measurement counts (counts dictionary, e.g., {'0': 600, '1': 424}).
        
        # Compute probability of measuring '1'
        p1 = counts.get('1', 0)/shots   #counts.get('1', 0) → how many times qubit 1 was measured as 1.   Divide by shots → probability of output being 1.
        pred = p1  #This probability is treated as the prediction
        print(f"Input: {xi}, Label: {yi}, Pred: {pred:.2f}")
        
        # Dummy gradient step (not real gradient)
        params -= learning_rate * (pred - yi) * 0.01  # very simple update  #(pred - yi) → difference between predicted probability and true label.
    print("Params:", params)  # Multiply by learning_rate * 0.01 → very simple update to parameters.


Epoch 1
Input: [0 0], Label: 0, Pred: 0.29
Input: [0 1], Label: 1, Pred: 0.72
Input: [1 0], Label: 1, Pred: 0.81
Input: [1 1], Label: 0, Pred: 0.17
Params: [0.34989997 0.98897381 0.09212578 0.14564073]
Epoch 2
Input: [0 0], Label: 0, Pred: 0.32
Input: [0 1], Label: 1, Pred: 0.69
Input: [1 0], Label: 1, Pred: 0.84
Input: [1 1], Label: 0, Pred: 0.19
Params: [0.34985896 0.9889328  0.09208477 0.14559972]
Epoch 3
Input: [0 0], Label: 0, Pred: 0.32
Input: [0 1], Label: 1, Pred: 0.71
Input: [1 0], Label: 1, Pred: 0.81
Input: [1 1], Label: 0, Pred: 0.19
Params: [0.34982576 0.9888996  0.09205157 0.14556651]
Epoch 4
Input: [0 0], Label: 0, Pred: 0.28
Input: [0 1], Label: 1, Pred: 0.70
Input: [1 0], Label: 1, Pred: 0.83
Input: [1 1], Label: 0, Pred: 0.19
Params: [0.34983943 0.98891327 0.09206524 0.14558019]
Epoch 5
Input: [0 0], Label: 0, Pred: 0.31
Input: [0 1], Label: 1, Pred: 0.70
Input: [1 0], Label: 1, Pred: 0.79
Input: [1 1], Label: 0, Pred: 0.16
Params: [0.34988435 0.98895819 0.09211016 0.